In [5]:
# Make sure these libraries are installed
# pip install matplotlib pandas plotly

import pandas as pd
import plotly.graph_objects as go # <-- Import Plotly
from datetime import datetime, timedelta

# Import our custom service functions
from oanda_api_service import get_oanda_client, get_historical_candles, get_closed_trades
from technical_indicator_service import add_bollinger_bands
from live_trader_config import CANDLE_GRANULARITY

def plot_recent_trades_plotly(num_trades_to_plot=10):
    """
    Fetches and plots the most recently closed trades using Plotly for interactive charts.
    """
    api_client = get_oanda_client()
    if not api_client:
        print("Failed to get OANDA client.")
        return

    print(f"Fetching the last {num_trades_to_plot} closed trades...")
    closed_trades = get_closed_trades(api_client, count=num_trades_to_plot)
    
    if not closed_trades:
        print("No closed trades found.")
        return

    print(f"Found {len(closed_trades)} recent closed trades to plot.")

    for trade_data in reversed(closed_trades):
        instrument = trade_data['instrument']
        entry_time = pd.to_datetime(trade_data['openTime'])
        exit_time = pd.to_datetime(trade_data['closeTime'])
        entry_price = float(trade_data['price'])
        exit_price = float(trade_data.get('averageClosePrice', trade_data['price']))
        pnl = float(trade_data.get('realizedPL', 0.0))
        units = float(trade_data['initialUnits'])
        
        print(f"\n--- Plotting trade on {instrument} opened at: {entry_time.strftime('%Y-%m-%d %H:%M:%S')} ---")

        plot_start_time = entry_time - timedelta(minutes=15 * 100)
        plot_end_time = exit_time + timedelta(minutes=15 * 50)

        df_candles = get_historical_candles(
            api_client, instrument, count=500, granularity=CANDLE_GRANULARITY
        )
        df_plot_data = df_candles[(df_candles.index >= plot_start_time) & (df_candles.index <= plot_end_time)]
        
        if df_plot_data.empty:
            continue
            
        df_with_bb = add_bollinger_bands(df_plot_data)

        # --- Create Plotly Figure ---
        fig = go.Figure()

        # 1. Add Candlestick Trace
        fig.add_trace(go.Candlestick(
            x=df_with_bb.index,
            open=df_with_bb['open'],
            high=df_with_bb['high'],
            low=df_with_bb['low'],
            close=df_with_bb['close'],
            name='Candles'
        ))

        # 2. Add Bollinger Bands Traces
        fig.add_trace(go.Scatter(x=df_with_bb.index, y=df_with_bb['bb_upper'], mode='lines', name='Upper BB', line={'dash': 'dash', 'color': 'cyan'}, opacity=0.7))
        fig.add_trace(go.Scatter(x=df_with_bb.index, y=df_with_bb['bb_middle'], mode='lines', name='Middle BB', line={'dash': 'dash', 'color': 'yellow'}, opacity=0.7))
        fig.add_trace(go.Scatter(x=df_with_bb.index, y=df_with_bb['bb_lower'], mode='lines', name='Lower BB', line={'dash': 'dash', 'color': 'cyan'}, opacity=0.7,
                                 fill='tonexty', fillcolor='rgba(0, 176, 246, 0.1)')) # Fill between lower and middle

        # 3. Add Trade Markers
        entry_color = 'green' if units > 0 else 'red'
        entry_symbol = 'triangle-up' if units > 0 else 'triangle-down'
        
        fig.add_trace(go.Scatter(
            x=[entry_time], y=[entry_price],
            mode='markers', name='Entry',
            marker=dict(color=entry_color, symbol=entry_symbol, size=12, line=dict(width=1, color='white'))
        ))
        
        fig.add_trace(go.Scatter(
            x=[exit_time], y=[exit_price],
            mode='markers', name='Exit',
            marker=dict(color='purple', symbol='square', size=10, line=dict(width=1, color='white'))
        ))

        # 4. Update Layout and Formatting
        pnl_color = 'lightgreen' if pnl >= 0 else 'lightcoral'
        title_str = f"Trade on {instrument} | P/L: <span style='color:{pnl_color};'>${pnl:.2f}</span>"
        
        fig.update_layout(
            title=title_str,
            yaxis_title='Price',
            xaxis_title='Date',
            template='plotly_dark',  # Use a dark theme suitable for finance
            xaxis_rangeslider_visible=False, # Hide the range slider
            legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1)
        )
        
        fig.show()


# --- USAGE EXAMPLE ---
plot_recent_trades_plotly(num_trades_to_plot=5)

Fetching the last 5 closed trades...
Found 5 recent closed trades to plot.

--- Plotting trade on USD_JPY opened at: 2025-06-19 06:30:06 ---



--- Plotting trade on USD_CAD opened at: 2025-06-19 06:30:15 ---



--- Plotting trade on GBP_USD opened at: 2025-06-19 08:30:07 ---



--- Plotting trade on EUR_JPY opened at: 2025-06-19 11:00:11 ---



--- Plotting trade on USD_CAD opened at: 2025-06-19 13:45:09 ---
